# VCT regression from the Kedro pipeline (simple and semiempirical rudder)

In [ ]:
%load_ext kedro.extras.extensions.ipython
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
#%config Completer.use_jedi = False

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from vessel_manoeuvring_models.prime_system import PrimeSystem
from phd.visualization.plot_prediction import plot_total_force, plot_force_components, predict, plot_compare_model_forces, plot_parameter_contributions
from vct.bokeh_plotting import create_tab
from vessel_manoeuvring_models.symbols import *
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify
plt.style.use('bmh')
plt.rcParams["figure.figsize"] = (15,10)
from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]
import statsmodels.api as sm

In [ ]:
#%reload_kedro
ship='wPCC'
ship_data = catalog.load(f"{ship}.ship_data")
model_loaders = catalog.load(f"{ship}.models_VCT")
models_rudder_VCT_loaders = catalog.load(f"{ship}.models_rudder_VCT")

df_VCT = catalog.load(f"{ship}.df_VCT")
resistance_MDL = catalog.load(f"{ship}.resistance_MDL")
tests_ek_smooth = catalog.load(f"{ship}.tests_ek_smooth")
tests_ek = catalog.load(f"{ship}.tests_ek")

In [ ]:
model = model_loaders['semiempirical_covered']()

In [ ]:
model_rudder_VCT = models_rudder_VCT_loaders['Abkowitz']()

In [ ]:
prime_system_ship = PrimeSystem(L=ship_data['L']*ship_data['scale_factor'], rho=df_VCT.iloc[0]['rho'])
keys = ['x0',
        'y0',
        'psi',
        'u',
        'v',
        'r',
        'beta',
        'V',
        'U',
        'delta',
        'thrust',
        'thrust_port',
        'thrust_stbd',
        'twa',
        'tws',
        'rev',
        'X_D',
        'Y_D',
        'N_D',
        'X_H',
        'Y_H',
        'N_H',
        'X_R',
        'Y_R',
        'N_R',
        'X_R_port',
        'Y_R_port',
        'N_R_port',
        'X_R_stbd',
        'Y_R_stbd',
        'N_R_stbd',
        'fx',
        'fy',
        'mz',
        'test type',
        'model_name',
       ]



df_VCT_prime = prime_system_ship.prime(df_VCT[keys], U=df_VCT['U'])
df_VCT_scaled = model.prime_system.unprime(df_VCT_prime, U=df_VCT['U']/np.sqrt(ship_data['scale_factor']))

In [ ]:
model.parameters

In [ ]:
model_rudder_VCT.parameters

In [ ]:
model_rudder_VCT.show_subsystems()

In [ ]:
rudders = model_rudder_VCT.subsystems['rudders']
rudder_hull_interaction = model.subsystems['rudder_hull_interaction']
rudder_hull_interaction.equations['Y_RHI'].subs(Y_R, rudders.equations['Y_R'].rhs).subs([
    (v,0),
    (r,0),
])

In [ ]:
rudder_hull_interaction.equations['Y_RHI'].subs(Y_RHI,Y_D_)

In [ ]:
rudders = model_rudder_VCT.subsystems['rudders']
rudder_hull_interaction = model.subsystems['rudder_hull_interaction']
rudder_hull_interaction.equations['N_RHI'].subs(Y_R, rudders.equations['Y_R'].rhs).subs([
    (v,0),
    (r,0),
])

In [ ]:
#model_ = model.copy()
#model_rudder_VCT_ = model_rudder_VCT.copy()

In [ ]:
#model.parameters['C_D_tune']=1.25
#model.parameters['C_D0_tune']=4.8

#a_H_ = 0.13
#model_.parameters['a_H'] = a_H_
#model_rudder_VCT_.parameters['a_H'] = a_H_

df_VCT_predict = predict(model, data=df_VCT_scaled)
df_VCT_predict['model_name'] = 'VCT regression semiempirical rudder'

df_VCT_predict_rudder_VCT = predict(model_rudder_VCT, data=df_VCT_scaled)
df_VCT_predict_rudder_VCT['model_name'] = 'VCT regression simple rudder'


df_result = pd.concat((df_VCT_scaled, df_VCT_predict,df_VCT_predict_rudder_VCT), axis=0)

In [ ]:
create_tab(df_result, ys=['X_D','Y_D','N_D','Y_H','N_H','X_R','Y_R','N_R'], plot_width=1200)

## Compare with MDL inverse dynamics

In [ ]:
#id = 22773
id = 22772
#id = 22770
#id = 22764
#id = 22774
#id = 22616  # Spiral


data_MDL = tests_ek_smooth[f'{id}']()
data_MDL_ek = tests_ek[f'{id}']()


data_MDL['V'] = data_MDL['U'] = np.sqrt(data_MDL['u']**2 + data_MDL['v']**2)
data_MDL['beta'] = -np.arctan2(data_MDL['v'],data_MDL['u'])
data_MDL['rev'] = data_MDL[['Prop/PS/Rpm','Prop/SB/Rpm']].mean(axis=1)
data_MDL['twa']=0
data_MDL['tws']=0
data_MDL['theta']=0
data_MDL['q']=0
data_MDL['phi'] = data_MDL['roll']
data_MDL['p'] = 0
data_MDL['q1d'] = 0
data_MDL['thrust_port'] = data_MDL['Prop/PS/Thrust']
data_MDL['thrust_stbd'] = data_MDL['Prop/SB/Thrust']

data_MDL_ek['V'] = data_MDL_ek['U'] = np.sqrt(data_MDL_ek['u']**2 + data_MDL_ek['v']**2)
data_MDL_ek['beta'] = -np.arctan2(data_MDL_ek['v'],data_MDL_ek['u'])
data_MDL_ek['rev'] = data_MDL_ek[['Prop/PS/Rpm','Prop/SB/Rpm']].mean(axis=1)
data_MDL_ek['twa']=0
data_MDL_ek['tws']=0
data_MDL_ek['theta']=0
data_MDL_ek['q']=0
data_MDL_ek['phi'] = data_MDL_ek['roll']
data_MDL_ek['p'] = 0
data_MDL_ek['q1d'] = 0
data_MDL_ek['thrust_port'] = data_MDL_ek['Prop/PS/Thrust']
data_MDL_ek['thrust_stbd'] = data_MDL_ek['Prop/SB/Thrust']

#start = (data_MDL['delta'].abs() > np.deg2rad(0.1)).idxmax()
#data_MDL = data_MDL.loc[start:].copy()
#data_MDL.index-=data_MDL.index[0]


In [ ]:
fig,axes=plt.subplots(nrows=2)
ax=axes[0]
data_MDL.plot(y='r1d',ax=ax,lw=0.5)
data_MDL_ek.plot(y='r1d',ax=ax,lw=0.5)

ax=axes[1]
data_MDL.plot(y='v1d',ax=ax,lw=0.5)
data_MDL_ek.plot(y='v1d',ax=ax,lw=0.5)

In [ ]:
model_MDL_loaders = catalog.load(f"{ship}.models_VCT_MDL")

In [ ]:
model = model_MDL_loaders['semiempirical_covered']()

In [ ]:
df_forces = predict(model=model, data=data_MDL)
df_forces_rudder_VCT = predict(model=model_rudder_VCT, data=data_MDL)

forces_from_motions = model.forces_from_motions(data=data_MDL)

In [ ]:
styles = {
    'Experiment':{'style':'-', 'color':'green', 'zorder':-10, 'lw':1.0,'label':'Experiment'},
    'Semi-empirical rudder':{'style':'b-','lw':0.5,'label':'Semi-empirical rudder'},
    'Simple rudder':{'style':'r-','lw':0.5,'label':'Simple rudder'},
    }

In [ ]:
#model.parameters['Nr'] = -0.0007225879175745149*1.7
#model.parameters['l_R'] = -3.1115000000000004*1.2
#model_rudder_VCT.parameters['Nr'] = -0.0019861528594665493*1.2

In [ ]:
model_rudder_VCT.parameters

In [ ]:
models = {
    'Semi-empirical rudder': model,
    'Simple rudder': model_rudder_VCT,
    
}

In [ ]:
plot_compare_model_forces(model=models, data=data_MDL_ek, keys=['N_D','N_H','N_R'], styles=styles)

In [ ]:
plot_parameter_contributions(model=model, data=data_MDL, prefix='N', unit='moment')

In [ ]:
plot_parameter_contributions(model=model_rudder_VCT, data=data_MDL, prefix='N', unit='moment')

In [ ]:
plot_compare_model_forces(model=models, data=data_MDL_ek, keys=['Y_D','Y_H','Y_R'], styles=styles)

In [ ]:
plot_parameter_contributions(model=model, data=data_MDL, prefix='Y', unit='force')

In [ ]:
plot_compare_model_forces(model=models, data=data_MDL, keys=['X_D','X_H','X_R'], styles=styles)

## Simulation

In [ ]:
result_MDL = model.simulate(data_MDL)
result_MDL_rudder_VCT = model_rudder_VCT.simulate(data_MDL)

In [ ]:
dataframes = {'Experiment':data_MDL,
              'Semiempirical rudder':result_MDL,
              'Simple rudder':result_MDL_rudder_VCT,
              
             }


ax = track_plots(dataframes=dataframes, lpp=ship_data['L'], beam=ship_data['B'], N=2, styles=styles);
plot(dataframes=dataframes, keys=['u','thrust','r','beta','delta','psi'], styles=styles);

## Investigating the added masses $N_{\dot{r}}$ and $Y_{\dot{r}}$

In [ ]:
eq = model.N_eq
eq

In [ ]:
eq_Nrdot_inverse = sp.Eq(p.Nrdot, sp.solve(eq,p.Nrdot)[0])
eq_Nrdot_inverse

In [ ]:
eq_ = eq_Nrdot_inverse.subs([
    (v1d,'v1d'),
    (r1d,'r1d'),
    (p.Nvdot,'Nvdot'),
])
lambda_Nrdot_inverse = lambdify(eq_.rhs, substitute_functions=True)

In [ ]:
prediction = predict(model=model, data=data_MDL)

In [ ]:
prediction['Nrdot'] = run(lambda_Nrdot_inverse, inputs=prediction, **model.ship_parameters, **model.parameters)

In [ ]:
mask = prediction['r1d'].abs() > prediction['r1d'].abs().quantile(0.4)
prediction_ = prediction.loc[mask]
prediction_['Nrdot'].loc[mask].hist(bins=100)

## Many

In [ ]:
ids = [
    22773,
    22772,
    22770,
    22764,
]

data_MDL_many = []
for id in ids:
    data_MDL = tests_ek[f'{id}']()
    data_MDL['V'] = data_MDL['U'] = np.sqrt(data_MDL['u']**2 + data_MDL['v']**2)
    data_MDL['beta'] = -np.arctan2(data_MDL['v'],data_MDL['u'])
    data_MDL['rev'] = data_MDL[['Prop/PS/Rpm','Prop/SB/Rpm']].mean(axis=1)
    data_MDL['twa']=0
    data_MDL['tws']=0
    data_MDL['theta']=0
    data_MDL['q']=0
    data_MDL['phi'] = data_MDL['roll']
    data_MDL['p'] = 0
    data_MDL['q1d'] = 0
    data_MDL['thrust_port'] = data_MDL['Prop/PS/Thrust']
    data_MDL['thrust_stbd'] = data_MDL['Prop/SB/Thrust']
    data_MDL_many.append(data_MDL)
    
data_MDL_many = pd.concat(data_MDL_many, axis=0)

In [ ]:
prediction = predict(model=model, data=data_MDL_many)

mask = prediction['r1d'].abs() > prediction['r1d'].abs().quantile(0.4)
prediction_ = prediction.loc[mask]

In [ ]:
eq = model.N_eq
move=p.Nrdot*r1d
eq_added_masses = sp.Eq(sp.solve(eq,move)[0],move)
eq_added_masses

eq_ = eq_added_masses.subs([
    (v1d,'v1d'),
    (r1d,'r1d'),
    (p.Nvdot,'Nvdot'),
])
lambda_y = lambdify(eq_.lhs, substitute_functions=True)

y = run(lambda_y, inputs=prediction_, Nvdot=0, **model.ship_parameters)
X = prediction_[['r1d']]

ols_model = sm.OLS(y,X)
fit3 = ols_model.fit()
fit3.summary2()

In [ ]:
Nrdot_denominator = run(lambdify(df_parameters.loc['Nrdot','denominator']),inputs=model.ship_parameters)
Nvdot_denominator = run(lambdify(df_parameters.loc['Nvdot','denominator']),inputs=model.ship_parameters)
Yrdot_denominator = run(lambdify(df_parameters.loc['Yrdot','denominator']),inputs=model.ship_parameters)
Yvdot_denominator = run(lambdify(df_parameters.loc['Yvdot','denominator']),inputs=model.ship_parameters)


In [ ]:
fit3.params['r1d']/Nrdot_denominator

In [ ]:
model.parameters['Nrdot']

In [ ]:
eq = model.Y_eq
move=p.Yvdot*v1d
eq_added_masses = sp.Eq(sp.solve(eq,move)[0],move)
eq_added_masses

eq_ = eq_added_masses.subs([
    (v1d,'v1d'),
    (r1d,'r1d'),
    (p.Yrdot,'Yrdot'),
])
lambda_y = lambdify(eq_.lhs, substitute_functions=True)

y = run(lambda_y, inputs=prediction_, Yrdot=0, **model.ship_parameters)
X = prediction_[['v1d']]

ols_model = sm.OLS(y,X)
fit_yvdot = ols_model.fit()
fit_yvdot.summary2()

In [ ]:
fit_yvdot.params['v1d']/Yvdot_denominator

In [ ]:
model.parameters['Yvdot']